# Toronto Postal Codes

### Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import geocoder

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

#!conda install -c conda-forge folium --yes
import folium

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries loaded.')

Libraries loaded.


## Question #1

### 1.1 Retrieve data

In [2]:
# get data from the Wikipedia page
list_of_dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

toronto_df = list_of_dfs[0]
#toronto_df = toronto_df.rename(columns={'Postal Code' : 'PostalCode'})
toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### 1.2 Remove Not Assigned Boroughs

In [3]:
toronto_df.drop(toronto_df[toronto_df['Borough'] == 'Not assigned'].index, inplace = True) 
toronto_df.reset_index(inplace=True, drop=True)
toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### 1.3 Check Not Assigned Neighbourhood

In [4]:
toronto_df[toronto_df['Neighbourhood'] == 'Not assigned']

,Postal Code,Borough,Neighbourhood


There is NO **Not Assigned** Neighbourhood

### 1.4 Final result. Dataframe Shape

In [5]:
toronto_df.shape

(103, 3)

The dataframe hase **103 rows** and **3 columns**

## Question #2

### Populate the Latitude and Longitude of postal code dataframe

In [6]:
def get_location_remote(row) :
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(row['Postal Code']))
        lat_lng_coords = g.latlng

    row['Latitude'] = lat_lng_coords[0]
    row['Longitude'] = lat_lng_coords[1]
    
    return row

# Use CSV
toronto_geo = pd.read_csv('https://cocl.us/Geospatial_data')
toronto_df = toronto_df.merge(toronto_geo, on='Postal Code')
toronto_df.head()

# User geocoder
#toronto_df['Latitude', 'Longitude'] = 
#toronto_df = toronto_df.apply(get_location_remote, axis=1)
#toronto_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Question #3

Quickly examine the resulting dataframe.

In [7]:
toronto_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [8]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df['Borough'].unique()),
        toronto_df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Toronto City.


In [9]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.6534817, -79.3839347.


#### Create a map of Toronto with neighborhoods superimposed on top.


In [10]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.


#### Define Foursquare Credentials and Version


In [11]:
CLIENT_ID = 'Y0P44AF55JQIYQ5Q24KAQ0X4MQRM0LCUAPSZ2OQ4MKL2A1GA' # your Foursquare ID
CLIENT_SECRET = 'BW5TXHHN1QJNA5T3EHJTOPV2GDDBJAVG1WAGAL2MWR0WB01S' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Y0P44AF55JQIYQ5Q24KAQ0X4MQRM0LCUAPSZ2OQ4MKL2A1GA
CLIENT_SECRET:BW5TXHHN1QJNA5T3EHJTOPV2GDDBJAVG1WAGAL2MWR0WB01S


### Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto


In [12]:
def getNearbyVenues(post_codes, boroughs, neighbourhoods, latitudes, longitudes, radius=500):
    venues_list=[]
    for post_code, borough, neighborhood, lat, lng in zip(post_codes, boroughs, neighbourhoods, latitudes, longitudes):
        print(post_code)

        # create the API request URL and params
        url = 'https://api.foursquare.com/v2/venues/explore'    
        params = dict(
        client_id=CLIENT_ID,
        client_secret=CLIENT_SECRET,
        v=VERSION,
        ll='{}, {}'.format(lat, lng),
        limit=LIMIT,
        radius=radius
        )
        
            
        # make the GET request
        results = requests.get(url=url, params=params).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            post_code, 
            borough,
            neighborhood,
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postal Code', 
                  'Borough',
                  'Neighbourhood',
                  'Postal Code Latitude', 
                  'Postal Code Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [13]:
# type your answer here
toronto_venues = getNearbyVenues(toronto_df['Postal Code'], toronto_df['Borough'], toronto_df['Neighbourhood'], toronto_df['Latitude'], toronto_df['Longitude'], radius=2000)


M3A
M4A
M5A
M6A
M7A
M9A
M1B
M3B
M4B
M5B
M6B
M9B
M1C
M3C
M4C
M5C
M6C
M9C
M1E
M4E
M5E
M6E
M1G
M4G
M5G
M6G
M1H
M2H
M3H
M4H
M5H
M6H
M1J
M2J
M3J
M4J
M5J
M6J
M1K
M2K
M3K
M4K
M5K
M6K
M1L
M2L
M3L
M4L
M5L
M6L
M9L
M1M
M2M
M3M
M4M
M5M
M6M
M9M
M1N
M2N
M3N
M4N
M5N
M6N
M9N
M1P
M2P
M4P
M5P
M6P
M9P
M1R
M2R
M4R
M5R
M6R
M7R
M9R
M1S
M4S
M5S
M6S
M1T
M4T
M5T
M1V
M4V
M5V
M8V
M9V
M1W
M4W
M5W
M8W
M9W
M1X
M4X
M5X
M8X
M4Y
M7Y
M8Y
M8Z


#### Let's check the size of the resulting dataframe


In [14]:
print(toronto_venues.shape)
toronto_venues.head()

(8599, 9)


,Postal Code,Borough,Neighbourhood,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,North York,Parkwoods,43.753259,-79.329656,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,M3A,North York,Parkwoods,43.753259,-79.329656,Donalda Golf & Country Club,43.752816,-79.342741,Golf Course
2,M3A,North York,Parkwoods,43.753259,-79.329656,Island Foods,43.745866,-79.346035,Caribbean Restaurant
3,M3A,North York,Parkwoods,43.753259,-79.329656,Graydon Hall Manor,43.763923,-79.342961,Event Space
4,M3A,North York,Parkwoods,43.753259,-79.329656,Tim Hortons,43.760668,-79.326368,Café


Let's check how many venues were returned for each neighborhood


In [15]:
toronto_venues.groupby('Postal Code').count()

,Borough,Neighbourhood,Postal Code Latitude,Postal Code Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postal Code,,,,,,,,
M1B,45,45,45,45,45,45,45,45
M1C,39,39,39,39,39,39,39,39
M1E,39,39,39,39,39,39,39,39
M1G,64,64,64,64,64,64,64,64
M1H,100,100,100,100,100,100,100,100
M1J,76,76,76,76,76,76,76,76
M1K,58,58,58,58,58,58,58,58
M1L,90,90,90,90,90,90,90,90
M1M,40,40,40,40,40,40,40,40


#### Let's find out how many unique categories can be curated from all the returned venues


In [16]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 329 uniques categories.


### Analyze Each Neighborhood


In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Postal Code'] = toronto_venues['Postal Code'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Postal Code,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Sout

And let's examine the new dataframe size.

In [18]:
toronto_onehot.shape

(8599, 330)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [19]:
toronto_grouped = toronto_onehot.groupby('Postal Code').mean().reset_index()
toronto_grouped

,Postal Code,Accessories Store,Afghan Restaurant,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Caucasian Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Cycle Studio,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Leather Goods Store,Light Rail Station,Lighting Store,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Medical Center,Medical Supply Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Nudist Beach,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Print Shop,Pub,Racecourse,Racetrack,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Restaurant,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,Sout

#### Let's confirm the new size

In [20]:
toronto_grouped.shape

(103, 330)

#### Let's print each neighborhood along with the top 5 most common venues


In [21]:
num_top_venues = 5

for hood in toronto_grouped['Postal Code']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Postal Code'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                  venue  freq
0           Zoo Exhibit  0.33
1  Fast Food Restaurant  0.07
2           Pizza Place  0.04
3                   Zoo  0.04
4           Gas Station  0.04


----M1C----
            venue  freq
0     Coffee Shop  0.08
1  Breakfast Spot  0.08
2            Park  0.05
3            Bank  0.05
4  Sandwich Place  0.05


----M1E----
                  venue  freq
0           Pizza Place  0.13
1                  Bank  0.05
2         Train Station  0.05
3  Fast Food Restaurant  0.05
4           Coffee Shop  0.05


----M1G----
                  venue  freq
0           Coffee Shop  0.14
1  Fast Food Restaurant  0.09
2        Sandwich Place  0.06
3        Discount Store  0.05
4           Pizza Place  0.05


----M1H----
            venue  freq
0     Coffee Shop  0.10
1     Gas Station  0.06
2  Clothing Store  0.05
3      Restaurant  0.05
4  Sandwich Place  0.04


----M1J----
                  venue  freq
0           Coffee Shop  0.09
1  Fast Food Restaurant  0.09


#### Let's put that into a _pandas_ dataframe
First, let's write a function to sort the venues in descending order.



In [22]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postal Code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
toronto_venues_sorted = pd.DataFrame(columns=columns)
toronto_venues_sorted['Postal Code'] = toronto_grouped['Postal Code']

for ind in np.arange(toronto_grouped.shape[0]):
    toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

toronto_venues_sorted.head()

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Zoo Exhibit,Fast Food Restaurant,Zoo,Athletics & Sports,Gas Station,Restaurant,Pizza Place,Fried Chicken Joint,Skating Rink,Caribbean Restaurant
1,M1C,Coffee Shop,Breakfast Spot,Pizza Place,Pharmacy,Pet Store,Park,Sandwich Place,Bank,Ice Cream Shop,Burger Joint
2,M1E,Pizza Place,Breakfast Spot,Bank,Fast Food Restaurant,Train Station,Coffee Shop,Supermarket,Bar,Greek Restaurant,Beer Store
3,M1G,Coffee Shop,Fast Food Restaurant,Sandwich Place,Discount Store,Pizza Place,Bank,Supermarket,Beer Store,Indian Restaurant,Chinese Restaurant
4,M1H,Coffee Shop,Gas Station,Clothing Store,Restaurant,Sandwich Place,Fast Food Restaurant,Gym,Department Store,Pharmacy,Bank


### Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [24]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Postal Code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 2, 2, 2, 1, 2, 2, 2, 2, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [25]:
# add clustering labels
toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_merged with toronto_df to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(toronto_venues_sorted.set_index('Postal Code'), on='Postal Code')

toronto_merged.head(10) # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,1,Coffee Shop,Chinese Restaurant,Japanese Restaurant,Gas Station,Discount Store,Pharmacy,Fried Chicken Joint,Sandwich Place,Supermarket,Pizza Place
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,Coffee Shop,Fast Food Restaurant,Gym,Sandwich Place,Japanese Restaurant,Middle Eastern Restaurant,Clothing Store,Grocery Store,Gas Station,Department Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Park,Restaurant,Café,Diner,Pub,Farmers Market,Gastropub,Bakery,Japanese Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Coffee Shop,Furniture / Home Store,Fast Food Restaurant,Grocery Store,Restaurant,Bank,Vietnamese Restaurant,Dessert Shop,Greek Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Hotel,Café,Japanese Restaurant,Ramen Restaurant,Pizza Place,Dance Studio,Park,Sushi Restaurant,Supermarket
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242,1,Pharmacy,Coffee Shop,Bank,Grocery Store,Park,Café,Golf Course,Liquor Store,Pizza Place,Shopping Mall
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,4,Zoo Exhibit,Fast Food Restaurant,Zoo,Athletics & Sports,Gas Station,Restaurant,Pizza Place,Fried Chicken Joint,Skating Rink,Caribbean Restaurant
7,M3B,North York,Don Mills,43.745906,-79.352188,1,Coffee Shop,Restaurant,Bank,Park,Pizza Place,Japanese Restaurant,Supermarket,Burger Joint,Italian Restaurant,Café
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,2,Pizza Place,Park,Skating Rink,Sandwich Place,Coffee Shop,Ice Cream Shop,Grocery Store,Fast Food Restaurant,Beer Store,Brewery
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Café,Restaurant,Gastropub,Japanese Restaurant,Pizza Place,Ramen Restaurant,Thai Restaurant,Park,Plaza


Finally, let's visualize the resulting clusters


In [26]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


#### Cluster 1


In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,0,Coffee Shop,Park,Restaurant,Café,Diner,Pub,Farmers Market,Gastropub,Bakery,Japanese Restaurant
4,Downtown Toronto,0,Coffee Shop,Hotel,Café,Japanese Restaurant,Ramen Restaurant,Pizza Place,Dance Studio,Park,Sushi Restaurant,Supermarket
9,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Gastropub,Japanese Restaurant,Pizza Place,Ramen Restaurant,Thai Restaurant,Park,Plaza
14,East York,0,Park,Coffee Shop,Pizza Place,Café,Gastropub,Thai Restaurant,Skating Rink,Grocery Store,Ice Cream Shop,Ethiopian Restaurant
15,Downtown Toronto,0,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Thai Restaurant,Gastropub,Pizza Place,Farmers Market,Plaza
16,York,0,Coffee Shop,Italian Restaurant,Bank,Café,Mexican Restaurant,Indian Restaurant,Grocery Store,Caribbean Restaurant,Pizza Place,Park
19,East Toronto,0,Coffee Shop,Pub,Beach,Breakfast Spot,Bakery,Ice Cream Shop,Park,Japanese Restaurant,BBQ Joint,Bar
20,Downtown Toronto,0,Coffee Shop,Café,Hotel,Restaurant,Japanese Restaurant,Plaza,Beer Bar,Farmers Market,Park,Dessert Shop
23,East York,0,Coffee Shop,Indian Restaurant,Bakery,Park,Grocery Store,Restaurant,Turkish Restaurant,Supermarket,Sushi Restaurant,Burger Joint
24,Downtown Toronto,0,Coffee Shop,Café,Pizza Place,Restaurant,Gastropub,Ramen Restaurant,Japanese Restaurant,Diner,Sushi Restaurant,Supermarket


#### Cluster 2


In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Coffee Shop,Chinese Restaurant,Japanese Restaurant,Gas Station,Discount Store,Pharmacy,Fried Chicken Joint,Sandwich Place,Supermarket,Pizza Place
3,North York,1,Clothing Store,Coffee Shop,Furniture / Home Store,Fast Food Restaurant,Grocery Store,Restaurant,Bank,Vietnamese Restaurant,Dessert Shop,Greek Restaurant
5,Etobicoke,1,Pharmacy,Coffee Shop,Bank,Grocery Store,Park,Café,Golf Course,Liquor Store,Pizza Place,Shopping Mall
7,North York,1,Coffee Shop,Restaurant,Bank,Park,Pizza Place,Japanese Restaurant,Supermarket,Burger Joint,Italian Restaurant,Café
10,North York,1,Clothing Store,Coffee Shop,Furniture / Home Store,Bank,Fried Chicken Joint,Fast Food Restaurant,Restaurant,Grocery Store,Sushi Restaurant,Boutique
13,North York,1,Coffee Shop,Restaurant,Japanese Restaurant,Park,Pizza Place,Middle Eastern Restaurant,Gym,Sandwich Place,Beer Store,Bank
17,Etobicoke,1,Coffee Shop,Golf Course,Bank,Grocery Store,Baseball Field,College Rec Center,Gas Station,Chinese Restaurant,Liquor Store,Clothing Store
26,Scarborough,1,Coffee Shop,Gas Station,Clothing Store,Restaurant,Sandwich Place,Fast Food Restaurant,Gym,Department Store,Pharmacy,Bank
27,North York,1,Coffee Shop,Bank,Chinese Restaurant,Sandwich Place,Pharmacy,Sushi Restaurant,Park,Pizza Place,Bakery,Japanese Restaurant
28,North York,1,Coffee Shop,Park,Pizza Place,Pharmacy,Deli / Bodega,Convenience Store,Gas Station,Bank,Restaurant,Sandwich Place


#### Cluster 3


In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,2,Coffee Shop,Fast Food Restaurant,Gym,Sandwich Place,Japanese Restaurant,Middle Eastern Restaurant,Clothing Store,Grocery Store,Gas Station,Department Store
8,East York,2,Pizza Place,Park,Skating Rink,Sandwich Place,Coffee Shop,Ice Cream Shop,Grocery Store,Fast Food Restaurant,Beer Store,Brewery
11,Etobicoke,2,Coffee Shop,Convenience Store,Park,Grocery Store,Pizza Place,Bank,Restaurant,Pharmacy,Fish & Chips Shop,Sushi Restaurant
12,Scarborough,2,Coffee Shop,Breakfast Spot,Pizza Place,Pharmacy,Pet Store,Park,Sandwich Place,Bank,Ice Cream Shop,Burger Joint
18,Scarborough,2,Pizza Place,Breakfast Spot,Bank,Fast Food Restaurant,Train Station,Coffee Shop,Supermarket,Bar,Greek Restaurant,Beer Store
21,York,2,Italian Restaurant,Sandwich Place,Bank,Furniture / Home Store,Bakery,Coffee Shop,Grocery Store,Pizza Place,Fast Food Restaurant,Mexican Restaurant
22,Scarborough,2,Coffee Shop,Fast Food Restaurant,Sandwich Place,Discount Store,Pizza Place,Bank,Supermarket,Beer Store,Indian Restaurant,Chinese Restaurant
29,East York,2,Park,Coffee Shop,Sandwich Place,Restaurant,Bakery,Pizza Place,Indian Restaurant,Japanese Restaurant,Grocery Store,Pharmacy
32,Scarborough,2,Fast Food Restaurant,Coffee Shop,Pizza Place,Sandwich Place,Pharmacy,Grocery Store,Gym,Liquor Store,Beer Store,Bank
38,Scarborough,2,Chinese Restaurant,Fast Food Restaurant,Grocery Store,Pharmacy,Coffee Shop,Train Station,Beer Store,Bank,Sandwich Place,Sporting Goods Shop


#### Cluster 4


In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,Scarborough,3,Farm,Golf Course,Sculpture Garden,Trail,Playground,Zoo Exhibit,Electronics Store,Escape Room,Ethiopian Restaurant,Event Service


#### Cluster 5

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Scarborough,4,Zoo Exhibit,Fast Food Restaurant,Zoo,Athletics & Sports,Gas Station,Restaurant,Pizza Place,Fried Chicken Joint,Skating Rink,Caribbean Restaurant
